## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [88]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [89]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [90]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [91]:
enlace

'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210407.ods'

In [92]:
try:
    datos_row = pd.read_excel(enlace,sheet_name='Comunicación')
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210407.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [93]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas con al menos 1 dosis,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,1283580,168000,564800,2016380,1622189,0.804506,1093523,528666,2021-04-06
1,Aragón,250695,35300,88700,374695,276558,0.738088,178668,97890,2021-04-06
2,Asturias,241925,32700,68000,342625,264498,0.771975,169720,94778,2021-04-06
3,Baleares,136080,18000,78000,232080,181442,0.781808,130499,50943,2021-04-06
4,Canarias,276090,35800,145200,457090,339311,0.742329,229083,110228,2021-04-06
5,Cantabria,115395,15300,38800,169495,127290,0.750996,82329,44961,2021-04-06
6,Castilla y Leon,555465,78300,159900,793665,610573,0.769308,390292,220281,2021-04-06
7,Castilla La Mancha,356715,49600,136400,542715,423319,0.780002,284704,138615,2021-04-06
8,Cataluña,1220930,168400,519100,1908430,1504694,0.788446,1062206,442488,2021-04-06
9,C. Valenciana,711550,96800,337500,1145850,926076,0.808200,663216,262860,2021-04-06


..hacemos una copia para hacerle las transformaciones necesarias:

In [94]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [95]:
datos_hoy.columns

Index(['Unnamed: 0', 'Dosis entregadas Pfizer (1)',
       'Dosis entregadas Moderna (1)', 'Dosis entregadas AstraZeneca (1)',
       'Total Dosis entregadas (1)', 'Dosis administradas (2)',
       '% sobre entregadas', 'Nº Personas con al menos 1 dosis',
       'Nº Personas vacunadas(pauta completada)',
       'Fecha de la última vacuna registrada (2)'],
      dtype='object')

In [96]:
datos_hoy.columns=['ccaa',
                   'Dosis entregadas Pfizer',
                   'Dosis entregadas Moderna',
                   'Dosis entregadas AstraZeneca',
                   'Dosis entregadas',
                   'Dosis administradas',
                   '% sobre entregadas',
                   'Total 1 vacuna',
                   'Total pauta completada',
                   'Fecha de la ultima vacuna registrada']

In [97]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [98]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [99]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [100]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,1283580,168000,564800,2016380,1622189,0.804506,1093523,528666,6/4/21,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,250695,35300,88700,374695,276558,0.738088,178668,97890,6/4/21,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,241925,32700,68000,342625,264498,0.771975,169720,94778,6/4/21,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,136080,18000,78000,232080,181442,0.781808,130499,50943,6/4/21,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,276090,35800,145200,457090,339311,0.742329,229083,110228,6/4/21,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,115395,15300,38800,169495,127290,0.750996,82329,44961,6/4/21,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,555465,78300,159900,793665,610573,0.769308,390292,220281,6/4/21,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,356715,49600,136400,542715,423319,0.780002,284704,138615,6/4/21,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,1220930,168400,519100,1908430,1504694,0.788446,1062206,442488,6/4/21,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,711550,96800,337500,1145850,926076,0.808200,663216,262860,6/4/21,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [101]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas','Total 1 vacuna',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [102]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [103]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla',
       'Fuerzas Armadas', 'Totales'], dtype=object)

In [104]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,7/4/21,Andalucía,1283580,168000,564800,2016380,1622189,0.804506,1093523,528666,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,7/4/21,Aragón,250695,35300,88700,374695,276558,0.738088,178668,97890,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,7/4/21,Asturias,241925,32700,68000,342625,264498,0.771975,169720,94778,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,7/4/21,Baleares,136080,18000,78000,232080,181442,0.781808,130499,50943,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,7/4/21,Canarias,276090,35800,145200,457090,339311,0.742329,229083,110228,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,7/4/21,Cantabria,115395,15300,38800,169495,127290,0.750996,82329,44961,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,7/4/21,Castilla y Leon,555465,78300,159900,793665,610573,0.769308,390292,220281,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,7/4/21,Castilla La Mancha,356715,49600,136400,542715,423319,0.780002,284704,138615,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,7/4/21,Cataluña,1220930,168400,519100,1908430,1504694,0.788446,1062206,442488,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,7/4/21,C. Valenciana,711550,96800,337500,1145850,926076,0.808200,663216,262860,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [105]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [106]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [107]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,6/4/21,País Vasco,314550.0,52900.0,148200.0,515650,441562,0.856321,325432.0,116130.0,NaN,5/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1306,6/4/21,Ceuta,9000.0,1300.0,5800.0,16100,13813,0.857950,10178.0,3635.0,NaN,5/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1307,6/4/21,Melilla,9000.0,1200.0,5800.0,16000,11584,0.724000,9093.0,2491.0,NaN,5/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1308,6/4/21,Fuerzas Armadas,19500.0,0.0,66100.0,85600,54450,0.636098,49423.0,5027.0,NaN,5/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [108]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [109]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total 1 vacuna', 'Total pauta completada',
       'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [110]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas','Total 1 vacuna',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [111]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total 1 vacuna,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,7/4/21,País Vasco,314550.0,52900.0,148200.0,515650,445819,0.864577,329676.0,116143.0,NaN,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1327,7/4/21,Ceuta,9000.0,1300.0,5800.0,16100,14358,0.891801,10719.0,3639.0,NaN,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1328,7/4/21,Melilla,9000.0,1200.0,5800.0,16000,11582,0.723875,9092.0,2490.0,NaN,5/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1329,7/4/21,Fuerzas Armadas,19500.0,0.0,66100.0,85600,58061,0.678283,53029.0,5032.0,NaN,6/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [112]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales',
       'Fuerzas Armadas'], dtype=object)

y volvemos a subirlo..

In [113]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

# Institucionalizados

In [207]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_institucionalizados_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Institucionalizados'

In [208]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [116]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210407.ods


In [117]:
datos_row

,Unnamed: 0,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA
0,Andalucía,41651,40929,38976,0.982665,0.935776
1,Aragón,18400,18154,17356,0.986630,0.943261
2,Asturias,11948,11617,11149,0.972297,0.933127
3,Baleares,5496,5122,4916,0.931951,0.894469
4,Canarias,10605,10601,9471,0.999623,0.893069
5,Cantabria,7470,7078,6534,0.947523,0.874699
6,Castilla y Leon,46000,45882,44168,0.997435,0.960174
7,Castilla - La Mancha,25566,24510,22929,0.958695,0.896855
8,Cataluña,75649,74073,63275,0.979167,0.836429
9,C. Valenciana,29500,29486,27116,0.999525,0.919186


In [118]:
datos_hoy = datos_row.copy()

In [119]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [120]:
datos_hoy.columns = ['ccaa', 'Denominador Poblacional CCAA',
       'Persona institucionalizada con al menos 1 dosis (2)',
       'Persona institucionalizada  Pauta Completa (2) ',
       '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
       '% Pautas Completas sobre Denominador Poblacional CCAA', 'date_pub',
       'source_name', 'source']

In [121]:
datos_hoy

,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,date_pub,source_name,source
0,Andalucía,41651,40929,38976,0.982665,0.935776,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,18400,18154,17356,0.986630,0.943261,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,11948,11617,11149,0.972297,0.933127,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,5496,5122,4916,0.931951,0.894469,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,10605,10601,9471,0.999623,0.893069,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,7470,7078,6534,0.947523,0.874699,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,46000,45882,44168,0.997435,0.960174,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla - La Mancha,25566,24510,22929,0.958695,0.896855,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,75649,74073,63275,0.979167,0.836429,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,29500,29486,27116,0.999525,0.919186,7/4/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [122]:
datos_hoy = datos_hoy[['date_pub','ccaa', 'Denominador Poblacional CCAA',
       'Persona institucionalizada con al menos 1 dosis (2)',
       'Persona institucionalizada  Pauta Completa (2) ',
       '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
       '% Pautas Completas sobre Denominador Poblacional CCAA',
       'source_name', 'source']]

In [123]:
datos_hoy

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,7/4/21,Andalucía,41651,40929,38976,0.982665,0.935776,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,7/4/21,Aragón,18400,18154,17356,0.986630,0.943261,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,7/4/21,Asturias,11948,11617,11149,0.972297,0.933127,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,7/4/21,Baleares,5496,5122,4916,0.931951,0.894469,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,7/4/21,Canarias,10605,10601,9471,0.999623,0.893069,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,7/4/21,Cantabria,7470,7078,6534,0.947523,0.874699,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,7/4/21,Castilla y Leon,46000,45882,44168,0.997435,0.960174,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,7/4/21,Castilla - La Mancha,25566,24510,22929,0.958695,0.896855,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,7/4/21,Cataluña,75649,74073,63275,0.979167,0.836429,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,7/4/21,C. Valenciana,29500,29486,27116,0.999525,0.919186,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [124]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_20210407'

In [125]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

..y ahora lo merge a los acumulados::

In [209]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [210]:
datos_acumulados

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,31/3/21,Andalucía,41651,40770,38848,0.978848,0.932703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,17829,17829,17025,1.000000,0.954905,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,11948,11577,11115,0.968949,0.930281,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,5496,5117,4869,0.931041,0.885917,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,10531,10531,9441,1.000000,0.896496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
115,7/4/21,Navarra,8343,8057,7864,0.965720,0.942587,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
116,7/4/21,País Vasco,23000,22924,20333,0.996696,0.884043,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
117,7/4/21,Ceuta,210,195,189,0.928571,0.900000,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
118,7/4/21,Melilla,315,305,292,0.968254,0.926984,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [128]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [129]:
datos_acumulados

,date_pub,ccaa,Denominador Poblacional CCAA,Persona institucionalizada con al menos 1 dosis (2),Persona institucionalizada Pauta Completa (2),% Con al menos 1 dosis sobre Denominador Poblacional CCAA,% Pautas Completas sobre Denominador Poblacional CCAA,source_name,source
0,31/3/21,Andalucía,41651,40770,38848,0.978848,0.932703,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,17829,17829,17025,1.000000,0.954905,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,11948,11577,11115,0.968949,0.930281,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,5496,5117,4869,0.931041,0.885917,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,10531,10531,9441,1.000000,0.896496,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...
115,7/4/21,Navarra,8343,8057,7864,0.965720,0.942587,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
116,7/4/21,País Vasco,23000,22924,20333,0.996696,0.884043,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
117,7/4/21,Ceuta,210,195,189,0.928571,0.900000,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
118,7/4/21,Melilla,315,305,292,0.968254,0.926984,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


y volvemos a subirlo:

In [212]:
nombre_fichero_acumulado

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_'

In [131]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

In [214]:
nombre_fichero_acumulado_long = '/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_institucionalizados_long'
pd.melt(datos_acumulados,id_vars=['date_pub','ccaa','source_name','source'],value_vars=[
    'Denominador Poblacional CCAA',
    'Persona institucionalizada con al menos 1 dosis (2)',
    'Persona institucionalizada  Pauta Completa (2) ',
    '% Con al menos 1 dosis sobre Denominador Poblacional CCAA',
    '% Pautas Completas sobre Denominador Poblacional CCAA']
                                      ).to_csv(nombre_fichero_acumulado_long,index=False)

# Etarios 1 dosis

In [197]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_etarios_1dosis_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=3,day=31)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Etarios_con_al_menos_1_dosis'

In [198]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [199]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210407.ods


In [136]:
datos_hoy = datos_row.copy()

In [137]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [138]:
datos_hoy.columns = ['ccaa', 'Personas con al menos 1 dosis >=80 años',
       'Población INE>=80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE', 'date_pub',
       'source_name', 'source']

In [139]:
datos_hoy = datos_hoy [['date_pub','ccaa', 'Personas con al menos 1 dosis >=80 años',
       'Población INE>=80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE', 
       'source_name', 'source']]

In [147]:
datos_hoy

,date_pub,ccaa,Personas con al menos 1 dosis >=80 años,Población INE>=80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,7/4/21,Andalucía,402738,413516,0.973936,77477,650381,0.119126,124728,916147,...,625846,0.042713,184,181921,0.001011,1093523,7062213,0.154841,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,7/4/21,Aragón,78591,98807,0.795399,3925,118955,0.032996,16047,156367,...,89180,0.063434,28,25106,0.001115,178668,1132764,0.157727,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,7/4/21,Asturias,80958,87826,0.921800,5634,109249,0.051570,29865,149652,...,54179,0.053379,30,15563,0.001928,169720,901209,0.188325,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,7/4/21,Baleares,41057,49542,0.828731,2975,80516,0.036949,20630,117231,...,85839,0.041764,33,23300,0.001416,130499,986279,0.132314,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,7/4/21,Canarias,63607,90236,0.704896,19237,154531,0.124486,49212,235092,...,163899,0.034393,34,44352,0.000767,229083,1871033,0.122437,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,7/4/21,Cantabria,36695,41042,0.894084,4896,53066,0.092262,5726,77345,...,34935,0.044397,5,10791,0.000463,82329,501384,0.164203,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,7/4/21,Castilla y Leon,180713,223657,0.807992,19814,245410,0.080738,45978,318575,...,144422,0.050082,91,40684,0.002237,390292,2092873,0.186486,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,7/4/21,Castilla - La Mancha,112269,135194,0.830429,7717,159101,0.048504,42029,218444,...,149518,0.038263,91,42364,0.002148,284704,1719964,0.165529,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,7/4/21,Cataluña,336495,447196,0.752455,44271,635292,0.069686,236047,837584,...,556316,0.061569,287,159864,0.001795,1062206,6531658,0.162624,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,7/4/21,C. Valenciana,254823,276531,0.921499,18307,437862,0.041810,87043,580728,...,352834,0.048652,111,103049,0.001077,663216,4269305,0.155345,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [200]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_1dosis_20210407'

In [153]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora merge con los acumulados:

In [201]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [202]:
#datos_acumulados.columns = datos_hoy.columns

In [203]:
datos_acumulados

,date_pub,ccaa,Personas con al menos 1 dosis >=80 años,Población INE>=80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,375382,413516,0.9077810773948287,21914,650381,0.03369409622974841,60757,916147,...,625846,0.03992036379556632,144,181921,0.0007915523771307326,933984,7062213,0.13225089642580873,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,71256,98807,0.7211634803202203,3553,118955,0.029868437644487413,11417,156367,...,89180,0.06159452792105853,28,25106,0.0011152712499004221,161061,1132764,0.14218407364640825,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,67037,87826,0.763293329993396,4258,109249,0.03897518512755266,15803,149652,...,54179,0.05197585780468447,30,15563,0.0019276489108783653,138822,901209,0.15403973994933473,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,33141,49542,0.6689475596463607,2346,80516,0.029137065924785138,15295,117231,...,85839,0.040972052330525754,31,23300,0.0013304721030042918,115375,986279,0.11698008372884346,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,54220,90236,0.600868832838335,15913,154531,0.10297610188247018,33435,235092,...,163899,0.034124674342125336,27,44352,0.0006087662337662338,199517,1871033,0.1066346772077243,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,7/4/21,País Vasco,104995,157950,0.6647356758467869,4447,213081,0.02086999779426603,74413,280119,...,140323,0.07885378733350912,6,41824,0.0001434583014537108,329676,1898123,0.1736852669716346,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
122,7/4/21,Ceuta,2094,2605,0.8038387715930903,1395,4318,0.32306623436776283,1800,7857,...,7581,0.036142989051576306,0,2175,0.0,10719,66158,0.1620212219232746,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
123,7/4/21,Melilla,1552,2349,0.6607066836951895,1791,3748,0.477854855923159,835,7827,...,8536,0.02225866916588566,6,2324,0.0025817555938037868,9092,66178,0.1373870470549125,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
124,7/4/21,Fuerzas Armadas,0,-,-,4,-,-,1423,-,...,-,-,0,-,-,53029,-,-,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [157]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [158]:
datos_acumulados

,date_pub,ccaa,Personas con al menos 1 dosis >=80 años,Población INE>=80 años,%,Personas con al menos 1 dosis 70-79 años,Población INE70-79 años,%.1,Personas con al menos 1 dosis 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas con al menos 1 dosis,Total Población INE Población a Vacunar (1),% Con al menos 1 dosis sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,375382,413516,0.9077810773948287,21914,650381,0.03369409622974841,60757,916147,...,625846,0.03992036379556632,144,181921,0.0007915523771307326,933984,7062213,0.13225089642580873,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,71256,98807,0.7211634803202203,3553,118955,0.029868437644487413,11417,156367,...,89180,0.06159452792105853,28,25106,0.0011152712499004221,161061,1132764,0.14218407364640825,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,67037,87826,0.763293329993396,4258,109249,0.03897518512755266,15803,149652,...,54179,0.05197585780468447,30,15563,0.0019276489108783653,138822,901209,0.15403973994933473,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,33141,49542,0.6689475596463607,2346,80516,0.029137065924785138,15295,117231,...,85839,0.040972052330525754,31,23300,0.0013304721030042918,115375,986279,0.11698008372884346,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,54220,90236,0.600868832838335,15913,154531,0.10297610188247018,33435,235092,...,163899,0.034124674342125336,27,44352,0.0006087662337662338,199517,1871033,0.1066346772077243,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,7/4/21,País Vasco,104995,157950,0.664736,4447,213081,0.020870,74413,280119,...,140323,0.078854,6,41824,0.000143,329676,1898123,0.173685,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
122,7/4/21,Ceuta,2094,2605,0.803839,1395,4318,0.323066,1800,7857,...,7581,0.036143,0,2175,0.000000,10719,66158,0.162021,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
123,7/4/21,Melilla,1552,2349,0.660707,1791,3748,0.477855,835,7827,...,8536,0.022259,6,2324,0.002582,9092,66178,0.137387,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
124,7/4/21,Fuerzas Armadas,0,-,-,4,-,-,1423,-,...,-,-,0,-,-,53029,-,-,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [161]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

In [206]:
nombre_fichero_acumulado_long = '/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_1dosis_long'
pd.melt(datos_acumulados,id_vars=['date_pub','ccaa','source_name','source'],value_vars=[
    'Personas con al menos 1 dosis >=80 años',
       'Población INE>=80 años', '%',
       'Personas con al menos 1 dosis 70-79 años', 'Población INE70-79 años',
       '%.1', 'Personas con al menos 1 dosis 60-69 años',
       'Población INE60-69 años', '%.2',
       'Personas con al menos 1 dosis 50-59 años', 'Población INE50-59 años',
       '%.3', 'Personas con al menos 1 dosis 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas con al menos 1 dosis',
       'Total Población INE Población a Vacunar (1)',
       '% Con al menos 1 dosis sobre Población a Vacunar INE']
                                      ).to_csv(nombre_fichero_acumulado_long,index=False)

# Etarios con pauta completa

In [162]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_etarios_pauta_completa_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

sheet_name = 'Etarios_con_pauta_completa'

In [163]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_GIV_comunicacion_{}.ods'
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

In [164]:
try:
    datos_row = pd.read_excel(enlace,sheet_name=sheet_name)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210407.ods


In [165]:
datos_hoy = datos_row.copy()

In [166]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [167]:
datos_hoy.columns = ['ccaa', 'Personas pauta completa >=80 años',
       'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE', 'date_pub',
       'source_name', 'source']

In [168]:
datos_hoy = datos_hoy[['date_pub','ccaa', 'Personas pauta completa >=80 años',
       'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE',
       'source_name', 'source']]

In [169]:
datos_hoy

,date_pub,ccaa,Personas pauta completa >=80 años,Población INE>=80 años,%,Personas pauta completa 70-79 años,Población INE70-79 años,%.1,Personas pauta completa 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas pauta completa,Total Población INE Población a Vacunar (1),% pauta completa sobre Población a Vacunar INE,source_name,source
0,7/4/21,Andalucía,257816,413516,0.623473,13815,650381,0.021241,42265,916147,...,625846,0.019054,85,181921,0.000467,528666,7062213,0.074858,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,7/4/21,Aragón,48140,98807,0.487212,3203,118955,0.026926,9433,156367,...,89180,0.016203,25,25106,0.000996,97890,1132764,0.086417,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,7/4/21,Asturias,44085,87826,0.501958,3644,109249,0.033355,9971,149652,...,54179,0.043190,25,15563,0.001606,94778,901209,0.105168,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,7/4/21,Baleares,17483,49542,0.352892,2179,80516,0.027063,4951,117231,...,85839,0.014399,30,23300,0.001288,50943,986279,0.051652,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,7/4/21,Canarias,34381,90236,0.381012,13081,154531,0.084650,9154,235092,...,163899,0.014424,19,44352,0.000428,110228,1871033,0.058913,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,7/4/21,Cantabria,23021,41042,0.560913,1673,53066,0.031527,3719,77345,...,34935,0.017060,3,10791,0.000278,44961,501384,0.089674,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,7/4/21,Castilla y Leon,116682,223657,0.521701,9334,245410,0.038034,20036,318575,...,144422,0.018065,82,40684,0.002016,220281,2092873,0.105253,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,7/4/21,Castilla - La Mancha,60130,135194,0.444768,5480,159101,0.034444,13473,218444,...,149518,0.022298,85,42364,0.002006,138615,1719964,0.080592,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,7/4/21,Cataluña,160027,447196,0.357845,26750,635292,0.042107,47524,837584,...,556316,0.028584,193,159864,0.001207,442488,6531658,0.067745,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,7/4/21,C. Valenciana,124001,276531,0.448416,9950,437862,0.022724,23007,580728,...,352834,0.014457,91,103049,0.000883,262860,4269305,0.061570,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [170]:
nombre_fichero_con_fecha

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_20210407'

In [171]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora los merge con los acumulados

In [177]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [178]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [179]:
#datos_acumulados.columns = datos_hoy.columns

In [180]:
datos_acumulados

,date_pub,ccaa,Personas pauta completa >=80 años,Población INE>=80 años,%,Personas pauta completa 70-79 años,Población INE70-79 años,%.1,Personas pauta completa 60-69 años,Población INE60-69 años,...,Población INE18-24 años,%.5,16-17 años,Población INE16-17 años,%.6,Total Personas pauta completa,Total Población INE Población a Vacunar (1),% pauta completa sobre Población a Vacunar INE,source_name,source
0,31/3/21,Andalucía,240454,413516,0.5814865688389325,12788,650381,0.019662321008762557,41807,916147,...,625846,0.018677118652192393,83,181921,0.0004562419951517417,507430,7062213,0.0718514154132706,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,31/3/21,Aragón,40245,98807,0.40730919874097987,3048,118955,0.025623134798873525,9273,156367,...,89180,0.015934065934065933,25,25106,0.0009957779016968055,89278,1132764,0.07881429847699963,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,31/3/21,Asturias,37353,87826,0.4253068567394621,3490,109249,0.031945372497688765,9709,149652,...,54179,0.04263644585540523,24,15563,0.0015421191287026924,87093,901209,0.09664018002483331,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,31/3/21,Baleares,12403,49542,0.2503532356384482,2088,80516,0.025932733866560684,4856,117231,...,85839,0.014317501368841669,30,23300,0.0012875536480686696,45509,986279,0.04614211597326923,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,31/3/21,Canarias,31183,90236,0.34557161221685356,11216,154531,0.07258090609651138,9016,235092,...,163899,0.014331997144582944,19,44352,0.0004283910533910534,104712,1871033,0.055964806606831626,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,7/4/21,País Vasco,45117,157950,0.285641,3504,213081,0.016444,11935,280119,...,140323,0.020325,4,41824,0.000096,116143,1898123,0.061188,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
122,7/4/21,Ceuta,1582,2605,0.607294,373,4318,0.086383,313,7857,...,7581,0.005804,0,2175,0.000000,3639,66158,0.055005,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
123,7/4/21,Melilla,861,2349,0.366539,57,3748,0.015208,269,7827,...,8536,0.006443,6,2324,0.002582,2490,66178,0.037626,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
124,7/4/21,Fuerzas Armadas,0,-,-,4,-,-,431,-,...,-,-,0,-,-,5032,-,-,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [183]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Personas pauta completa >=80 años',
       'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
       'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
       'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
       'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
       'Población INE25-49 años', '%.4', '18-24 años',
       'Población INE18-24 años', '%.5', '16-17 años',
       'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
       'Total Población INE Población a Vacunar (1)',
       '% pauta completa sobre Población a Vacunar INE', 'source_name',
       'source'],
      dtype='object')

In [181]:
nombre_fichero_acumulado

'/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_'

In [182]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)

In [192]:
nombre_fichero_acumulado_long = '/Users/mharias/documents/proyectos/covid/vacunacion/csv/estado_vacunacion_etarios_pauta_completa_long'
pd.melt(datos_acumulados,id_vars=['date_pub','ccaa','source_name','source'],value_vars=[
    'Personas pauta completa >=80 años',
    'Población INE>=80 años', '%', 'Personas pauta completa 70-79 años',
    'Población INE70-79 años', '%.1', 'Personas pauta completa 60-69 años',
    'Población INE60-69 años', '%.2', 'Personas pauta completa 50-59 años',
    'Población INE50-59 años', '%.3', 'Personas pauta completa 25-49 años',
    'Población INE25-49 años', '%.4', '18-24 años',
    'Población INE18-24 años', '%.5', '16-17 años',
    'Población INE16-17 años', '%.6', 'Total Personas pauta completa',
    'Total Población INE Población a Vacunar (1)',
    '% pauta completa sobre Población a Vacunar INE']
                                      ).to_csv(nombre_fichero_acumulado_long,index=False)